# Create a binary blob for an OAK-device from an object detection API model

This notebook is used to convert a TF model (more specifically one created using the object detection API) to a `.blob`-file used by OAK-devices.
There are two main steps to do this:
1. Convert the TF model into the OpenVINO intermediate representation (IR)
2. Compile the OpenVINO IR into a Myriad binary (`.blob`-file)

This notebook is based on these tutorials:
- https://docs.openvinotoolkit.org/latest/openvino_docs_MO_DG_prepare_model_convert_model_tf_specific_Convert_Object_Detection_API_Models.html
- https://docs.luxonis.com/en/latest/pages/tutorials/local_convert_openvino/#compile-the-model

## Options
You may need to change these according to your paths and the model used.

For the `TRANSFORMATION_CONFIG` have a look at the [OpenVINO documentation](https://docs.openvinotoolkit.org/latest/openvino_docs_MO_DG_prepare_model_convert_model_tf_specific_Convert_Object_Detection_API_Models.html).

In [1]:
# Options
import os
MODEL_DIR = "./exported_models/ssd_mobilenet_v2_fpnlite_320x320/"
SAVED_MODEL = os.path.join(MODEL_DIR, "saved_model")
PIPELINE_CONFIG = os.path.join(MODEL_DIR, "pipeline.config")

OPENVINO_DIR = "/opt/intel/openvino_2021"
TRANSFORMATION_CONFIG = os.path.join(OPENVINO_DIR, "deployment_tools/model_optimizer/extensions/front/tf/ssd_support_api_v2.4.json")

## Step 0: Install Prequisites (OpenVINO)
This assumes you are using Ubuntu (or any other distribution using the APT package manager, e.g. Debian).
For other ways to install OpenVINO refere to their [website](https://software.intel.com/content/www/us/en/develop/tools/openvino-toolkit/download.html)

### Add Intel OpenVINO GPG-key

In [ ]:
!curl https://apt.repos.intel.com/openvino/2021/GPG-PUB-KEY-INTEL-OPENVINO-2021 -o openvino.gpg
!sudo apt-key add openvino.gpg

### Add the repository

In [ ]:
!echo "deb https://apt.repos.intel.com/openvino/2021/ all main"| sudo tee /etc/apt/sources.list.d/intel-openvino-2021.list
!sudo apt update

### Install the package

In [ ]:
!sudo apt -y install intel-openvino-dev-ubuntu20-2021.4.582

### Install other OpenVINO dependencies/prerequisites

In [ ]:
%cd /opt/intel/openvino_2021/deployment_tools/model_optimizer/install_prerequisites
!./install_prerequisites.sh
%cd

## Step 1: Convert the model to OpenVINO IR

### Setup OpenVINO environment

In [ ]:
!source {OPENVINO_DIR}/bin/setupvars.sh

### Convert the model to OpenVINO IR
Use the `mo_tf.py` tool to convert a TF-model to OpenVINO IR.

This will generate three files: `saved_model.xml`, `saved_model.bin` and `saved_model.mapping`.

Options are:
- `saved_model_dir` should point to the `saved_model`-directory of the exported (frozen) model.
- `tensorflow_object_detection_api_pipeline_config` should point to the `pipeline.conf` file used to create the model.
- `transformation_config` points to a special config that helps the optimizer to convert the model. There are already some configs provided by OpenVINO. For more info check out [this](https://docs.openvinotoolkit.org/latest/openvino_docs_MO_DG_prepare_model_convert_model_tf_specific_Convert_Object_Detection_API_Models.html)
- `reverse_input_channels` is used to invert the order of the input channels, i.e. `RGB <-> BGR`.
This is required if the model was trained with one order and you want to use inference with the other

In [ ]:
!python3 {OPENVINO_DIR}/deployment_tools/model_optimizer/mo_tf.py --saved_model_dir {SAVED_MODEL} \
    --tensorflow_object_detection_api_pipeline_config {PIPELINE_CONFIG} \
    --transformations_config {TRANSFORMATION_CONFIG} \
    --reverse_input_channels

### Compile the IR to Myriad code for execution on the OAK-device
The OAK device cannot execute the OpenVINO IR directly so we have to compile it to a Myriad binary.

This takes as input the OpenVINO IR and generates a `saved_model.blob`-file.

In [ ]:
!{OPENVINO_DIR}/deployment_tools/tools/compile_tool/compile_tool -m saved_model.xml -ip U8 -d MYRIAD -VPU_NUMBER_OF_SHAVES 6 -VPU_NUMBER_OF_CMX_SLICES 6